In [6]:
!pip install beautifulsoup4

In [4]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.5 MB/s eta 0:00:00


In [105]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.1/943.1 kB 10.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.0 MB/s eta 0:00:00


In [106]:
from pymongo import MongoClient

In [1]:
import os, re
import pandas as pd
import codecs
from getpass import getpass
import imaplib, email
from bs4 import BeautifulSoup
from nltk import word_tokenize


In [16]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
import re
import traceback
from pymongo import MongoClient

# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

service=None

In [60]:
def getAutheticationAndConnection():
 # Variable creds will store the user access token.
 # If no valid token found, we will create one.
 creds = None

 # The file token.pickle contains the user access token.
 # Check if it exists
 if os.path.exists('token.pickle'):

  # Read the token from the file and store it in the variable creds
  with open('token.pickle', 'rb') as token:
   creds = pickle.load(token)

 # If credentials are not available or are invalid, ask the user to log in.
 if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
   creds.refresh(Request())
  else:
   flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
   creds = flow.run_local_server(port=0)

  # Save the access token in token.pickle file for the next run
  with open('token.pickle', 'wb') as token:
   pickle.dump(creds, token)

 # Connect to the Gmail API
 global service
 service = build('gmail', 'v1', credentials=creds)

In [90]:

def clean_email_content(subject, sender, body, received_date, redirect_link):
    """
    Cleans and formats email content for database storage with error handling.
    """
    print(body)
    try:
        # Clean subject
        clean_subject = subject.strip() if subject else None
        if not clean_subject:
            raise ValueError("Subject is missing or empty.")

        # Clean sender
        clean_sender = sender.strip() if sender else None
        if not clean_sender:
            raise ValueError("Sender is missing or empty.")

        # Clean body
        clean_body = re.sub(r"\s{2,}", " ", body).strip() if body else None
        if not clean_body:
            raise ValueError("Body is missing or empty.")

        # Clean received date
        clean_received_date = received_date.strip() if received_date else None
        if not clean_received_date:
            raise ValueError("Received date is missing or empty.")

        # Clean redirect link
        clean_redirect_link = redirect_link.strip() if redirect_link else None
        if not clean_redirect_link:
            raise ValueError("Redirect link is missing or empty.")

        # Construct a dictionary for structured data
        email_data = {
            "subject": clean_subject,
            "sender": clean_sender,
            "body": clean_body,
            "received_date": clean_received_date,
            "redirect_link": clean_redirect_link,
        }

        return email_data

    except Exception as e:
        # Log the error and the failing input
        traceback.print_exc()
        print(f"Error while cleaning email content: {e.printstack}")
        #print(f"Inputs:\n Subject: {subject}\n Sender: {sender}\n Body: {body}\n Received Date: {received_date}\n Redirect Link: {redirect_link}")
        return None

In [ ]:
# Function to connect to MongoDB and insert data
def store_to_mongodb(data, db_name="email_data", collection_name="emails"):
    """
    Stores cleaned email data to MongoDB.
    """
    try:
        # Connect to MongoDB (local)
        client = MongoClient("mongodb://localhost:27017/")
        db = client[db_name]
        collection = db[collection_name]

        # Insert the data
        result = collection.insert_one(data)
        print(f"Data inserted with ID: {result.inserted_id}")

    except Exception as e:
        print(f"Error while connecting to MongoDB: {e}")
        traceback.print_exc()

In [102]:
def getParsedContent(numberOfEmails):
    global service
    redirect_links = []
    result_emails=[]
    result = service.users().messages().list(userId='me',maxResults=min(50,numberOfEmails)).execute()
    messages = result.get('messages')
    
    numberOfEmails-=50
    while 'nextPageToken' in result and numberOfEmails>0:
        page_token = response['nextPageToken']
        response = service.users().messages().list(userId="me", maxResults=50, pageToken=page_token).execute()
        if 'messages' in response:
            messages.extend(response['messages'])
            numberOfEmails-=50
    for msg in messages:
    # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        redirect_link = f"https://mail.google.com/mail/u/0/#all/{msg['id']}"
        try:
            payload = txt['payload']
            headers = payload['headers']

            # Look for Subject and Sender Email in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
                if d['name'] == 'Date':
                    received_date= d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)

            # Now, the data obtained is in lxml. So, we will parse
            # it with BeautifulSoup library
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()

            # Printing the subject, sender's email and message
            cleaned_email = clean_email_content(subject, sender,str(body), received_date, redirect_link)
            print("Subject: ",cleaned_email['subject'])
            print("From: ", cleaned_email['sender'])
            print("Message: ",cleaned_email['body'])
            print("Recieved Date",cleaned_email['received_date'])
            print("Redirect link ",cleaned_email['redirect_link'])
            print('\n')
            result_emails.append(cleaned_email)
        except:
            pass
        

In [103]:
getAutheticationAndConnection()
getParsedContent(10)

[<p>Over the next seven days, we will show you how to get the most out of Perplexity and why it is the future of search.

͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏
  ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏
  ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏
  ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏
  ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏
  ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏
  ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏   ­͏  

In [44]:
result = service.users().messages().list(userId='me',maxResults=50).execute()
result

{'messages': [{'id': '193a39e4b43c644f', 'threadId': '193a39e4b43c644f'},
  {'id': '193a30a2090b0d9d', 'threadId': '193a2b0084eb0cc2'},
  {'id': '193a300de5e2130e', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2ffe61fe78e8', 'threadId': '193a2ffe61fe78e8'},
  {'id': '193a2fbda641d15e', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2fbd566b2bee', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2fb0a6e70ae6', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2f96d7a51ee3', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2ee5cb3c006f', 'threadId': '193a2ee5cb3c006f'},
  {'id': '193a2eaabeffd67f', 'threadId': '193a2b0084eb0cc2'},
  {'id': '193a2df58a37bed1', 'threadId': '193a2dd66bfc2e39'},
  {'id': '193a2de36d26ba56', 'threadId': '193a2de36d26ba56'},
  {'id': '193a2dd66bfc2e39', 'threadId': '193a2dd66bfc2e39'},
  {'id': '193a2d162ff7a54f', 'threadId': '193a2d162ff7a54f'},
  {'id': '193a2b0084eb0cc2', 'threadId': '193a2b0084eb0cc2'},
  {'id': '193a0fe98f43928d', 'threadId': '193a0fe98f43928d

In [41]:
txttemp=service.users().messages().get(userId='me', id=messages[1]['id']).execute()
txttemp

{'id': '193a300de5e2130e',
 'threadId': '193a2f96d7a51ee3',
 'labelIds': ['CATEGORY_PERSONAL', 'INBOX'],
 'snippet': 'Welcome to r/Microsoft! Please read this message in its entirety so that you are familiarized with how the subreddit works. Content which benefits the community (news, rumors, and ...',
 'payload': {'partId': '',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'sorgavasalidiots@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:7011:a1c3:b0:415:e955:5296 with SMTP id jd3csp1712284mdc;        Sat, 7 Dec 2024 13:22:43 -0800 (PST)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IGHvoeFLnLZA30wLKd8yrQaZ8n2bQdJYq2jO928YA47twVya/pS7L2YsrB6YvZt4AFfUYHf'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:620a:1b96:b0:7b6:6701:7a4a with SMTP id af79cd13be357-7b6bcbb2cfemr1160158885a.53.1733606563484;        Sat, 07 Dec 2024 13:22:43 -0800 (PST)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1733

In [42]:
try:
    payload = txttemp['payload']
    headers = payload['headers']
    # Look for Subject and Sender Email in the headers
    for d in headers:
        if d['name'] == 'Subject':
            subject = d['value']
        if d['name'] == 'From':
            sender = d['value']
        if d['name'] == 'Date':
            recievedDate= d['value']
    # The Body of the message is in Encrypted format. So, we have to decode it.
    # Get the data and decode it with base 64 decoder.
    parts = payload.get('parts')[0]
    data = parts['body']['data']
    
    data = data.replace("-","+").replace("_","/")
    
    decoded_data = base64.b64decode(data)
    
    # Now, the data obtained is in lxml. So, we will parse
    # it with BeautifulSoup library
    soup = BeautifulSoup(decoded_data , "lxml")
    body = soup.body()

    # Printing the subject, sender's email and message
    print("Subject: ", subject)
    print("From: ", sender)
    print("Message: ", body)
    print('\n')
except:
    pass

New message from u/welcomebot


In [ ]:
def getParsedContent(numberOfEmails):
    global service
    result = service.users().messages().list(userId='me',maxResults=50).execute()
    messages = result.get('messages')
    numberOfEmails-=50
    while 'nextPageToken' in response and numberOfEmails>0:
        page_token = response['nextPageToken']
        response = service.users().messages().list(userId="me", maxResults=50, pageToken=page_token).execute()
        if 'messages' in response:
            messages.extend(response['messages'])
            numberOfEmails-=50
    for msg in messages:
    # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        try:
            payload = txt['payload']
            headers = payload['headers']

            # Look for Subject and Sender Email in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
                if d['name'] == 'Date':
                    recievedDate= d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)

            # Now, the data obtained is in lxml. So, we will parse
            # it with BeautifulSoup library
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()

            # Printing the subject, sender's email and message
            print("Subject: ", subject)
            print("From: ", sender)
            print("Message: ", body)
            print("Recieved Date", recievedDate)
            print('\n')
        except:
            pass
        

In [ ]:
def getParsedContent():
    global service
    result = service.users().messages().list(userId='me').execute()
    messages = result.get('messages')
    for msg in messages:
    # Get the message from its id
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        try:
            payload = txt['payload']
            headers = payload['headers']

            # Look for Subject and Sender Email in the headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']
                if d['name'] == 'From':
                    sender = d['value']
                if d['name'] == 'Date':
                    recievedDate= d['value']

            # The Body of the message is in Encrypted format. So, we have to decode it.
            # Get the data and decode it with base 64 decoder.
            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)

            # Now, the data obtained is in lxml. So, we will parse
            # it with BeautifulSoup library
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()

            # Printing the subject, sender's email and message
            print("Subject: ", subject)
            print("From: ", sender)
            print("Message: ", body)
            print("Recieved Date", recievedDate)
            print('\n')
        except:
            pass
        

In [23]:
for msg in messages:
  # Get the message from its id
  txt = service.users().messages().get(userId='me', id=msg['id']).execute()
  

{'messages': [{'id': '193a30a2090b0d9d', 'threadId': '193a2b0084eb0cc2'},
  {'id': '193a300de5e2130e', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2ffe61fe78e8', 'threadId': '193a2ffe61fe78e8'},
  {'id': '193a2fbda641d15e', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2fbd566b2bee', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2fb0a6e70ae6', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2f96d7a51ee3', 'threadId': '193a2f96d7a51ee3'},
  {'id': '193a2ee5cb3c006f', 'threadId': '193a2ee5cb3c006f'},
  {'id': '193a2eaabeffd67f', 'threadId': '193a2b0084eb0cc2'},
  {'id': '193a2df58a37bed1', 'threadId': '193a2dd66bfc2e39'},
  {'id': '193a2de36d26ba56', 'threadId': '193a2de36d26ba56'},
  {'id': '193a2dd66bfc2e39', 'threadId': '193a2dd66bfc2e39'},
  {'id': '193a2d162ff7a54f', 'threadId': '193a2d162ff7a54f'},
  {'id': '193a2b0084eb0cc2', 'threadId': '193a2b0084eb0cc2'},
  {'id': '193a0fe98f43928d', 'threadId': '193a0fe98f43928d'},
  {'id': '1939d5e0d4ea793a', 'threadId': '1939ca29359fab60

In [12]:
# import the required libraries
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup

# Define the SCOPES. If modifying it, delete the token.pickle file.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

services=None
 
def getEmails():
 # Variable creds will store the user access token.
 # If no valid token found, we will create one.
 creds = None

 # The file token.pickle contains the user access token.
 # Check if it exists
 if os.path.exists('token.pickle'):

  # Read the token from the file and store it in the variable creds
  with open('token.pickle', 'rb') as token:
   creds = pickle.load(token)

 # If credentials are not available or are invalid, ask the user to log in.
 if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
   creds.refresh(Request())
  else:
   flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
   creds = flow.run_local_server(port=0)

  # Save the access token in token.pickle file for the next run
  with open('token.pickle', 'wb') as token:
   pickle.dump(creds, token)

 # Connect to the Gmail API
 service = build('gmail', 'v1', credentials=creds)

 # request a list of all the messages
 result = service.users().messages().list(userId='me').execute()

 # We can also pass maxResults to get any number of emails. Like this:
 # result = service.users().messages().list(maxResults=200, userId='me').execute()
 messages = result.get('messages')

 # messages is a list of dictionaries where each dictionary contains a message id.

 # iterate through all the messages
 for msg in messages:
  # Get the message from its id
  txt = service.users().messages().get(userId='me', id=msg['id']).execute()

  # Use try-except to avoid any Errors
  try:
   # Get value of 'payload' from dictionary 'txt'
   payload = txt['payload']
   headers = payload['headers']

   # Look for Subject and Sender Email in the headers
   for d in headers:
    if d['name'] == 'Subject':
     subject = d['value']
    if d['name'] == 'From':
     sender = d['value']

   # The Body of the message is in Encrypted format. So, we have to decode it.
   # Get the data and decode it with base 64 decoder.
   parts = payload.get('parts')[0]
   data = parts['body']['data']
   data = data.replace("-","+").replace("_","/")
   decoded_data = base64.b64decode(data)

   # Now, the data obtained is in lxml. So, we will parse
   # it with BeautifulSoup library
   soup = BeautifulSoup(decoded_data , "lxml")
   body = soup.body()

   # Printing the subject, sender's email and message
   print("Subject: ", subject)
   print("From: ", sender)
   print("Message: ", body)
   print('\n')
  except:
   pass


getEmails()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1063389508119-dnbq9jfen4r5ep4noa0obk5fvmuhkmag.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56769%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=Jgi4Xpg0nhdddQ3yztJ2fUVVGD1GEH&access_type=offline
Subject:  [Action Needed] OAuth Verification Request Acknowledgement
From:  API OAuth Dev Verification <api-oauth-dev-verification-reply+1o9ck74jhiv661d@google.com>
Message:  [<p>Appeal Received




Google Cloud Platform



MY CONSOLE




Notification



This is an automated email. Please ignore it if you have already responded  
to your prior submission.

If you have not yet responded, please read the information below carefully  
to determine the next step of your submission review.



Please respond to this email to begin the verification process. This  
verification request will be auto closed if we do not receive a reply.
